# Setup

In [4]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
#standard tools
import numpy as np
import pandas as pd

#transformations
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

#models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

#validation
from sklearn.metrics import accuracy_score

#model saving
from tensorflow.keras.models import save_model, load_model

#parameter tuning
from sklearn.model_selection import GridSearchCV

In [16]:
train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Train_Eng.csv', index_col=0)

In [9]:
train[train['GAME_ID'] == 20300001]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,H_ELO,A_ELO,H_LAST_3_PTS,A_LAST_3_PTS,H_LAST_3_FG_PCT,A_LAST_3_FG_PCT,H_LAST_3_FT_PCT,A_LAST_3_FT_PCT,H_LAST_3_FG3_PCT,A_LAST_3_FG3_PCT,H_LAST_3_AST,A_LAST_3_AST,H_LAST_3_REB,A_LAST_3_REB,H_LAST_5_PTS,A_LAST_5_PTS,H_LAST_5_FG_PCT,A_LAST_5_FG_PCT,H_LAST_5_FT_PCT,A_LAST_5_FT_PCT,H_LAST_5_FG3_PCT,A_LAST_5_FG3_PCT,H_LAST_5_AST,A_LAST_5_AST,H_LAST_5_REB,A_LAST_5_REB,H_LAST_10_PTS,A_LAST_10_PTS,H_LAST_10_FG_PCT,A_LAST_10_FG_PCT,H_LAST_10_FT_PCT,A_LAST_10_FT_PCT,H_LAST_10_FG3_PCT,A_LAST_10_FG3_PCT,H_LAST_10_AST,A_LAST_10_AST,H_LAST_10_REB,A_LAST_10_REB
0,2003-10-28,20300001,1610612755,1610612748,2003,1493.236369,1506.763631,79.0,86.0,0.329,0.352,0.897,0.647,0.143,0.222,7.0,15.0,44.0,55.0,79.0,86.0,0.329,0.352,0.897,0.647,0.143,0.222,7.0,15.0,44.0,55.0,79.0,86.0,0.329,0.352,0.897,0.647,0.143,0.222,7.0,15.0,44.0,55.0


In [17]:
X_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Train_Eng.csv', index_col=0)
X_test = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Test_Eng.csv', index_col=0)
y_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Train_Eng.csv', index_col=0)
y_test = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Test_Eng.csv', index_col=0)

In [5]:
X_train.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,H_ELO,A_ELO,H_LAST_3_PTS,A_LAST_3_PTS,H_LAST_3_FG_PCT,A_LAST_3_FG_PCT,H_LAST_3_FT_PCT,A_LAST_3_FT_PCT,H_LAST_3_FG3_PCT,A_LAST_3_FG3_PCT,H_LAST_3_AST,A_LAST_3_AST,H_LAST_3_REB,A_LAST_3_REB,H_LAST_5_PTS,A_LAST_5_PTS,H_LAST_5_FG_PCT,A_LAST_5_FG_PCT,H_LAST_5_FT_PCT,A_LAST_5_FT_PCT,H_LAST_5_FG3_PCT,A_LAST_5_FG3_PCT,H_LAST_5_AST,A_LAST_5_AST,H_LAST_5_REB,A_LAST_5_REB,H_LAST_10_PTS,A_LAST_10_PTS,H_LAST_10_FG_PCT,A_LAST_10_FG_PCT,H_LAST_10_FT_PCT,A_LAST_10_FT_PCT,H_LAST_10_FG3_PCT,A_LAST_10_FG3_PCT,H_LAST_10_AST,A_LAST_10_AST,H_LAST_10_REB,A_LAST_10_REB
0,2003-10-28,20300001,1610612755,1610612748,2003,1493.236369,1506.763631,79.0,86.0,0.329,0.352,0.897,0.6470,0.143,0.222,7.0,15.0,44.0,55.0,79.0,86.0,0.329,0.352,0.897,0.6470,0.143,0.222,7.0,15.0,44.0,55.0,79.0,86.0,0.329,0.352,0.897,0.6470,0.143,0.222,7.0,15.0,44.0,55.0
1,2003-10-29,20300008,1610612765,1610612754,2003,1508.918907,1505.084618,100.0,62.0,0.442,0.385,0.650,0.5760,0.365,0.300,20.5,11.0,38.5,30.0,100.0,62.0,0.442,0.385,0.650,0.5760,0.365,0.300,20.5,11.0,38.5,30.0,100.0,62.0,0.442,0.385,0.650,0.5760,0.365,0.300,20.5,11.0,38.5,30.0
2,2003-10-29,20300005,1610612752,1610612753,2003,1487.755097,1484.933987,86.0,93.5,0.380,0.454,0.852,0.7635,0.188,0.175,19.0,16.0,37.0,35.5,86.0,93.5,0.380,0.454,0.852,0.7635,0.188,0.175,19.0,16.0,37.0,35.5,86.0,93.5,0.380,0.454,0.852,0.7635,0.188,0.175,19.0,16.0,37.0,35.5
3,2003-10-29,20300010,1610612741,1610612764,2003,1494.915382,1506.288649,58.0,87.5,0.457,0.432,0.700,0.6720,0.667,0.420,10.0,20.5,31.0,43.0,58.0,87.5,0.457,0.432,0.700,0.6720,0.667,0.420,10.0,20.5,31.0,43.0,58.0,87.5,0.457,0.432,0.700,0.6720,0.667,0.420,10.0,20.5,31.0,43.0
4,2003-10-29,20300004,1610612738,1610612748,2003,1483.516974,1495.307736,89.0,80.0,0.462,0.380,0.538,0.7355,0.214,0.236,21.0,15.5,36.0,49.5,89.0,80.0,0.462,0.380,0.538,0.7355,0.214,0.236,21.0,15.5,36.0,49.5,89.0,80.0,0.462,0.380,0.538,0.7355,0.214,0.236,21.0,15.5,36.0,49.5


In [23]:
X_test.head()

AttributeError: ignored

# Transform

In [18]:
#drop rows not used by model
X_train.drop(columns=['GAME_DATE_EST', 'GAME_ID'], inplace=True)
X_test.drop(columns=['GAME_DATE_EST', 'GAME_ID'], inplace=True)

In [19]:
#change dtype of team_id to easily seperate them later for encoding
X_train['HOME_TEAM_ID'] = X_train['HOME_TEAM_ID'].astype('object')
X_train['VISITOR_TEAM_ID'] = X_train['VISITOR_TEAM_ID'].astype('object')
X_test['HOME_TEAM_ID'] = X_test['HOME_TEAM_ID'].astype('object')
X_test['VISITOR_TEAM_ID'] = X_test['VISITOR_TEAM_ID'].astype('object')

In [20]:
#convert y to np.array
y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)

In [21]:
#function that will take in X, sclaer and encoder. Returns fully transformed X as np.array
def transform_X(X_train, X_test, scaler, encoder): 
  enc_cols = X_train.select_dtypes(include=object).columns
  scale_cols = X_train.select_dtypes(exclude=object).columns

  #seperate numerical and categorical columns
  X_train_num = X_train[scale_cols]
  X_train_cat = X_train[enc_cols]

  #scale X_train
  X_train_scaled = scaler.fit_transform(X_train_num)

  #encode X_train team_id's 
  X_train_cat['HOME_TEAM_ID'] = enc.fit_transform(X_train_cat['HOME_TEAM_ID'])
  X_train_cat['VISITOR_TEAM_ID'] = enc.transform(X_train_cat['VISITOR_TEAM_ID'])

  #convert to dataframe and combine back with encoded categorical columns
  X_train_scaled = pd.DataFrame(X_train_scaled)

  #repeat the process with X_test, but this time, only transform (don't re-fit)
  X_train = X_train_scaled.merge(X_train_cat, on=X_train_scaled.index)
  X_train.drop('key_0', axis=1, inplace=True)
  X_train = np.array(X_train)

  X_test_num = X_test[scale_cols]
  X_test_cat = X_test[enc_cols]

  X_test_scaled = scaler.transform(X_test_num)
  X_test_cat['HOME_TEAM_ID'] = enc.transform(X_test_cat['HOME_TEAM_ID'])
  X_test_cat['VISITOR_TEAM_ID'] = enc.transform(X_test_cat['VISITOR_TEAM_ID'])

  X_test_scaled = pd.DataFrame(X_test_scaled)

  X_test = X_test_scaled.merge(X_test_cat, on=X_test_scaled.index)
  X_test.drop('key_0', axis=1, inplace=True)
  X_test = np.array(X_test)

  return X_train, X_test

In [22]:
#initialize scaler and encoder and pass them into function
scaler = StandardScaler()
enc = LabelEncoder()
X_train, X_test = transform_X(X_train, X_test, scaler, enc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [23]:
#check that shapes make sense. Should have 41 columns
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((23293, 41), (23293,), (58, 41), (58,))

In [ ]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,-1.641456,-0.169625,-0.040019,-2.327462,-1.596513,-3.537792,-2.904533,2.181889,-1.735855,-3.092057,-1.944543,-4.214882,-1.943988,0.358778,2.942944,-2.608797,-1.777737,-4.288501,-3.491308,2.633157,-2.083048,-3.911599,-2.447664,-4.839800,-2.226113,0.431817,3.480326,-2.914934,-1.982109,-5.328428,-4.330075,3.247160,-2.579314,-5.210267,-3.274884,-5.591438,-2.562317,0.520006,4.264194,18.0,11.0
1,-1.641456,-0.017955,-0.056325,-0.065677,-4.169273,-0.333306,-1.968071,-1.671012,-2.853451,0.202211,-0.785637,-0.369132,-3.078346,-0.932703,-2.981459,-0.078100,-4.655636,-0.406997,-2.364117,-2.014480,-3.424425,0.247235,-0.983378,-0.425897,-3.527507,-1.116328,-3.528474,-0.085660,-5.204626,-0.507556,-2.929816,-2.482744,-4.242177,0.324754,-1.311333,-0.490333,-4.062453,-1.359633,-4.311802,28.0,17.0
2,-1.641456,-0.222636,-0.252021,-1.573534,-0.792526,-2.091520,-0.010016,1.479943,0.097948,-2.424300,-2.642858,-0.796438,-1.660398,-1.284926,-1.678090,-1.765231,-0.878394,-2.536672,-0.007261,1.786422,0.117942,-3.068592,-3.329990,-0.916330,-1.900764,-1.538549,-1.986538,-1.971842,-0.975072,-3.152636,-0.002003,2.203251,0.149186,-4.088303,-4.458050,-1.057122,-2.187283,-1.872262,-2.425083,15.0,16.0
3,-1.641456,-0.153387,-0.044632,-4.589247,-1.435716,0.092068,-0.634324,-0.891072,-1.342335,4.683603,0.997295,-3.360271,-0.384245,-2.693815,0.099231,-5.139495,-1.597868,0.108247,-0.758722,-1.073663,-1.610733,5.904749,1.269370,-3.858933,-0.436696,-3.227434,0.116102,-5.744208,-1.780701,0.132383,-0.935509,-1.322844,-1.993799,7.854377,1.709516,-4.457859,-0.499630,-3.922778,0.147716,4.0,27.0
4,-1.641456,-0.263624,-0.151275,-1.250422,-2.239703,0.233859,-2.109960,-3.418076,-0.342794,-2.038484,-1.736534,-0.226697,-1.802193,-1.519740,1.639576,-1.403703,-2.497212,0.279995,-2.534903,-4.121911,-0.411051,-2.581521,-2.184843,-0.262419,-2.063438,-1.820030,1.938390,-1.567660,-2.787738,0.345696,-3.141977,-5.080919,-0.506591,-3.440057,-2.922452,-0.301403,-2.374800,-2.214015,2.377474,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23288,1.657602,-0.043190,1.185770,1.765292,0.869048,0.489084,-0.568109,1.453945,-1.190174,0.414904,-0.211137,0.627914,1.459087,-0.345666,1.679072,1.512625,1.316005,-0.468826,-0.471801,1.782658,-1.074182,0.007447,0.203069,-0.000854,1.417790,0.657002,2.134636,1.733159,2.207164,0.055590,0.392615,1.530509,-0.726745,-0.146470,-0.130685,0.643246,1.488051,0.690883,3.097858,16.0,12.0
23289,1.657602,0.141034,0.943681,0.436942,0.833316,-1.108433,-0.416762,0.471222,0.032362,-0.104462,-0.523150,-1.271222,1.080967,0.202235,1.916048,1.898255,1.363970,-0.516915,0.074716,1.440201,-0.314699,0.007447,0.026604,-0.916330,1.027372,0.431817,0.845017,2.366378,0.636187,-0.021203,-0.880347,1.537468,-0.537038,0.945574,-1.535379,-0.036901,0.100425,0.520006,0.662275,20.0,10.0
23290,1.657602,0.577658,0.371642,0.436942,0.261591,-0.352212,0.358893,-0.267121,0.625265,0.464368,0.526799,-0.226697,1.270027,-0.267395,-0.137745,0.958281,0.140862,-0.056631,-0.526453,-0.117793,0.066932,0.176048,0.165523,0.456884,0.962302,0.600706,0.676806,1.409814,1.012147,0.234773,0.609019,1.208057,1.121141,0.392072,0.402999,1.285608,1.150520,-0.163499,-0.332540,11.0,17.0
23291,1.657602,0.627705,0.645019,0.257435,0.976247,-1.136791,-0.227578,0.284036,2.031443,-0.104462,-0.419146,-0.986351,0.513788,1.141494,-1.875570,0.548549,0.836355,-1.382525,-1.066138,0.800445,2.398282,-0.045007,-0.806913,-0.654766,-0.013743,1.107371,-1.285658,0.817013,1.992330,-0.942714,-0.939752,0.804412,1.915099,0.003125,-0.483117,-0.717048,0.362949,0.930109,-1.224444,23.0,8.0


# Model XGB

In [ ]:
model = XGBClassifier(random_state=42)

In [ ]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.43478260869565216

In [ ]:
model.feature_importances_

array([0.02035807, 0.18539456, 0.15843667, 0.0222765 , 0.01541162,
       0.01896044, 0.01830961, 0.0176991 , 0.01437655, 0.01491439,
       0.01489838, 0.02140236, 0.01297543, 0.01525617, 0.01521123,
       0.01917601, 0.02043504, 0.01731728, 0.01089686, 0.01368929,
       0.0219085 , 0.01310356, 0.01258388, 0.01439979, 0.02088499,
       0.0172182 , 0.01085769, 0.01595918, 0.02023781, 0.02087307,
       0.01799138, 0.01925574, 0.01354102, 0.01492249, 0.01813431,
       0.01742251, 0.01952944, 0.01695332, 0.0137151 , 0.01651634,
       0.01659616], dtype=float32)

# Model LogReg

In [ ]:
model_logreg = LogisticRegression(random_state=120)

In [ ]:
model_logreg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=120, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred_logreg = model_logreg.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_logreg)

0.43478260869565216

# Model RandomForest

In [ ]:
model_rfc = RandomForestClassifier(random_state=120)

In [ ]:
model_rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=120,
                       verbose=0, warm_start=False)

In [ ]:
y_pred_rfc = model_rfc.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_rfc)

0.5217391304347826

# GridSearch

In [ ]:
# Create the parameter grid based on the results of grid search 
# Penalty type 
penalty = ['l1', 'l2', 'elasticnet', 'none']
# Solver type 
solver = ['lbfgs', 'liblinear']
# Maximum number of iterations 
max_iter = [int(x) for x in np.linspace(start = 80, stop = 120, num = 5)]
# l1 ratio 
l1_ratio = [0, 0.8, 0.9, 1]
# C
C = [0.5, 0.75, 1.0, 1.25, 1.5]
random_state=[120]
verbose=[1]

# Create the param grid
param_grid = {'penalty': penalty, 'solver': solver, 'max_iter':max_iter, 
    'random_state':random_state, 'verbose':verbose, 'l1_ratio':l1_ratio, 
    'C':C
}
print(param_grid)

# Instantiate the grid search model with 2-fold cross-validation
log_grid_search = GridSearchCV(estimator = LogisticRegression(random_state=120, verbose=1), param_grid = param_grid, cv = 5, n_jobs = -1)

{'penalty': ['l1', 'l2', 'elasticnet', 'none'], 'solver': ['lbfgs', 'liblinear'], 'max_iter': [80, 90, 100, 110, 120], 'random_state': [120], 'verbose': [1], 'l1_ratio': [0, 0.8, 0.9, 1], 'C': [0.5, 0.75, 1.0, 1.25, 1.5]}


In [ ]:
# Fit the grid search to the data
log_grid_search.fit(X_train, y_train)
best_log_grid = log_grid_search.best_estimator_
best_log_grid.fit(X_train, y_train)
y_pred_best_log = best_log_grid.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1501: UserWarning: l1_ratio parameter i

In [ ]:
accuracy_score(y_test, y_pred_best_log)

0.5576923076923077

# Model FCNN

In [ ]:
model_fcnn = Sequential()
model_fcnn.add(Dense(41, activation='relu', input_shape=(41,)))
model_fcnn.add(Dense(82, activation='relu'))
model_fcnn.add(Dense(164, activation='relu'))
model_fcnn.add(Dense(80, activation='relu'))
model_fcnn.add(Dense(40, activation='relu'))
model_fcnn.add(Dense(20, activation='relu'))
model_fcnn.add(Dense(10, activation='relu'))
model_fcnn.add(Dense(1, activation='sigmoid'))

model_fcnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_fcnn.fit(X_train, y_train, epochs=500, batch_size=64, verbose=1)

Epoch 1/500
364/364 [==============================] - 4s 3ms/step - loss: 0.6518 - accuracy: 0.6166
Epoch 2/500
364/364 [==============================] - 1s 2ms/step - loss: 0.6162 - accuracy: 0.6599
Epoch 3/500
364/364 [==============================] - 1s 2ms/step - loss: 0.6131 - accuracy: 0.6661
Epoch 4/500
364/364 [==============================] - 1s 2ms/step - loss: 0.6066 - accuracy: 0.6672
Epoch 5/500
364/364 [==============================] - 1s 2ms/step - loss: 0.6068 - accuracy: 0.6670
Epoch 6/500
364/364 [==============================] - 1s 3ms/step - loss: 0.6018 - accuracy: 0.6706
Epoch 7/500
364/364 [==============================] - 1s 2ms/step - loss: 0.5982 - accuracy: 0.6781
Epoch 8/500
364/364 [==============================] - 1s 2ms/step - loss: 0.6020 - accuracy: 0.6688
Epoch 9/500
364/364 [==============================] - 1s 2ms/step - loss: 0.5986 - accuracy: 0.6720
Epoch 10/500
364/364 [==============================] - 1s 3ms/step - loss: 0.5975 - accura

In [ ]:
loss, acc = model_fcnn.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {acc}')

2/2 [==============================] - 0s 6ms/step - loss: 4.8268 - accuracy: 0.5345
Test Accuracy: 0.5344827771186829


In [ ]:
acc_list = []
for i in range(X_test.shape[0]):
  loss, acc = model_fcnn.evaluate(X_test[:i+1], y_test[:i+1])
  acc_list.append(acc)

2/2 [==============================] - 0s 6ms/step - loss: 4.8268 - accuracy: 0.5345


In [ ]:
model_fcnn.save('drive/MyDrive/Colab Notebooks/Final/V2/FCNN_Model_Trained.h5')

In [24]:
loaded_model = load_model('drive/MyDrive/Colab Notebooks/Final/V2/FCNN_Model_Trained.h5')

In [25]:
loss, acc = loaded_model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: %.3f' % acc)

2/2 [==============================] - 3s 6ms/step - loss: 4.8268 - accuracy: 0.5345
Test Accuracy: 0.534


In [26]:
acc_list_loaded = []
for i in range(X_test.shape[0]):
  loss, acc = loaded_model.evaluate(X_test[:i+1], y_test[:i+1])
  acc_list_loaded.append(acc)

2/2 [==============================] - 0s 3ms/step - loss: 4.8268 - accuracy: 0.5345


# Plot Results

In [27]:
import matplotlib.pyplot as plt
import plotly.express as px

In [37]:
px.line(x=range(5, len(acc_list_loaded) - 30), y=acc_list_loaded[5:-30], title='Model Accuracy Over Time', labels={'x':'Number of Games Predicted', 'y':'Accuracy'})

In [ ]:
len(acc_list_loaded)

58